# ActBlue donor profile

In [9]:
import json
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import os

In [3]:
%matplotlib inline

plt.style.use('ggplot')

pd.options.display.float_format = '{:,.2f}'.format

In [7]:
def read_or_save(name, func):
    path = 'pickles/' + name + '.pickle.gz'
    if (os.path.isfile(path)):
        return pd.read_pickle(path)
    else:
        result = func()
        os.makedirs('pickles', exist_ok=True)
        result.to_pickle(path)
        return result

In [4]:
committee_id = "C00401224"
year_since = 2013

### Connect to the PostgreSQL database

In [5]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf['dbname'],
    user=conf['user'],
    host=conf['host'],
    password=conf['password']
)

### Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?

In [10]:
actblue_states = read_or_save(
    'actblue_states',
    lambda: pd.read_sql("""
    SELECT contributor_state,
           count(*),
           sum(contribution_amount)
    FROM fec_contributions
    WHERE filing_id IN
        (SELECT filing_id
         FROM fec_pac_summaries
         JOIN fec_amended_filings USING (filing_id)
         WHERE filer_committee_id_number = '""" + committee_id + """'
           AND extract(YEAR
                       FROM coverage_through_date) > """ + str(year_since) + """
         ORDER BY coverage_through_date DESC)
      AND form_type = 'SA11AI'
    GROUP BY contributor_state
    """, con=conn)
)
actblue_states.sort_values(by=['sum'], ascending=False)

,contributor_state,count,sum
30,CA,12062776,"283,218,125.72"
44,NY,5628188,"157,311,026.41"
50,MA,2773802,"72,470,951.55"
51,TX,3112392,"65,044,446.34"
1,FL,3159095,"64,572,158.63"
11,IL,2471078,"54,557,368.54"
32,WA,2741416,"51,188,999.37"
52,PA,2163765,"44,781,490.78"
37,VA,1627316,"44,353,311.58"
74,MD,1467096,"40,147,179.65"


### Questions to answer
> - Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?
> - Where is the money going? Are people giving to local candidates or are they giving to candidates around the country? Which campaigns/candidates/groups have done the best job tapping into this network?
> - What’s the gender breakdown and how does it compare to the gender breakdown of itemized contributions? Is it different?
> - Are these new donors?
> - Can we tell if these are recurring contributions or one time contributions?
> - Do the majority of these donors give repeatedly or are they one and done? How does that compare to itemized contributions? How “sticky” is this?
